<a href="https://colab.research.google.com/github/emadphysics/emadphysics-Discriminating_Cosmological_Scenarios_By_3DCNN/blob/main/3D_Convelotional_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

In [4]:
import os
import sys
import time
import numpy as np
import argparse
from argparse import RawTextHelpFormatter
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional import MaxPooling3D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from keras import callbacks
K.set_image_data_format('channels_first')
from sklearn.metrics import confusion_matrix


In [6]:
class History(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.valacc = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.valacc.append(logs.get('val_acc'))

In [28]:
class Cnn():
    def __init__(self):
        self.model = self.create_model(verbose=True)
    def create_model(self, verbose=False):
        model = Sequential()
        model.add(Conv3D(8, (2, 3, 10), input_shape=(1, 4, 5, 100),
                         padding="same", activation='relu'))
        model.add(Conv3D(8, (2, 3, 10), padding="same", activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 1, 5)))
        model.add(Conv3D(8, (2, 3, 10), padding="same", activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 1, 2)))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4, activation='softmax'))
        optimizer = Adam(lr=0.001, decay=0.001) 
        model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                      metrics=['accuracy'])
        if verbose:
            model.summary()
        return model

    def load_data(self, datapath):
        try:
            self.data = np.load(datapath)
        except Exception:
            print("Unable to load " + datapath)

    def reset(self, verbose=False):
        del self.model
        self.model = self.create_model(verbose)
        if verbose:
            print("Resetting CNN model.")

    def train(self, iterations, epochs):
        if not hasattr(self, 'data'):
            print("No data loaded.")
            return

        X = self.data
        X = X.reshape(1024, 1, 4, 5, 100)

        y = np.floor(np.linspace(0, 3.9999, 1024))
        y = np_utils.to_categorical(y)

        h = np.zeros((iterations, 2, epochs))

        for iteration in range(iterations):
            print("\nITERATION {}".format(iteration))
            print("-" * (10 + len(str(iteration))))
            i1 = np.arange(256)
            i2 = np.arange(256, 512)
            i3 = np.arange(512, 768)
            i4 = np.arange(768, 1024)

            np.random.shuffle(i1)
            np.random.shuffle(i2)
            np.random.shuffle(i3)
            np.random.shuffle(i4)

            i_train = np.concatenate((i1[64:], i2[64:], i3[64:], i4[64:]))
            i_test = np.concatenate((i1[:64], i2[:64], i3[:64], i4[:64]))
            np.random.shuffle(i_train)
            np.random.shuffle(i_test)

            X_train = X[i_train]
            y_train = y[i_train]
            X_test = X[i_test]
            y_test = y[i_test]
            
            history = History()
            self.model.fit(X_train, y_train, validation_data=(X_test, y_test),
                           epochs=epochs, batch_size=200, verbose=1,
                           callbacks=[history])
            h[iteration, 0] = history.losses
            h[iteration, 1] = history.valacc

            y_pred = self.model.predict(X_test)
            y_pred = np.argmax(y_pred, axis=1)
            y_test = np.argmax(y_test, axis=1)
            cm = confusion_matrix(y_test, y_pred) / 64.0



In [29]:
cnn = Cnn()
cnn.load_data('/content/gdrive/MyDrive/pdfs3d.npy')
cnn.train(iterations=5, epochs=200)



Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_75 (Conv3D)           (None, 8, 4, 5, 100)      488       
_________________________________________________________________
conv3d_76 (Conv3D)           (None, 8, 4, 5, 100)      3848      
_________________________________________________________________
max_pooling3d_50 (MaxPooling (None, 8, 4, 5, 20)       0         
_________________________________________________________________
conv3d_77 (Conv3D)           (None, 8, 4, 5, 20)       3848      
_________________________________________________________________
max_pooling3d_51 (MaxPooling (None, 8, 4, 5, 10)       0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 8, 4, 5, 10)       0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 1600)            